In [366]:

import types
import pandas as pd
from botocore.client import Config
import ibm_boto3

import json # library to handle JSON files

#!conda install -c conda-forge geopy --yes # uncomment this line if you haven't completed the Foursquare API lab
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

#!conda install -c conda-forge folium=0.5.0 --yes # uncomment this line if you haven't completed the Foursquare API lab
import folium # map rendering library

import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe
import numpy as np # library to handle data in a vectorized manner

# import k-means from clustering stage
from sklearn.cluster import KMeans

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

In [333]:
# The code was removed by Watson Studio for sharing.

,Postal Code,Borough,Neighborhood,Latitud,Longitud,State
0,20000,Aguascalientes,Zona Centro,21.8734,-102.2806,Aguascalientes
1,21000,Mexicali,Centro Cívico,31.7497,-115.1105,Baja California
2,22000,Tijuana,Zona Norte,32.4248,-116.8983,Baja California
3,22000,Tijuana,Zona Centro,32.4041,-116.8198,Baja California
4,22000,Tijuana,Zona Este,32.4248,-116.8983,Baja California


In [334]:
df_MetroZone = df_data_3.loc[(df_data_3['State']=='México') | (df_data_3['State']=='Distrito Federal')]

df_MetroZone.drop(['State'], axis=1, inplace = True)

df_data_2 = df_MetroZone[df_MetroZone['Postal Code']==2000]
df_data_2.head()

,Postal Code,Borough,Neighborhood,Latitud,Longitud
31,2000,Azcapotzalco,Centro de Azcapotzalco,19.4815,-99.1862


In [335]:
df_MetroZone.head()

,Postal Code,Borough,Neighborhood,Latitud,Longitud
31,2000,Azcapotzalco,Centro de Azcapotzalco,19.4815,-99.1862
32,4000,Coyoacán,Villa Coyoacán,19.3400,-99.1617
33,5000,Cuajimalpa de Morelos,Cuajimalpa,19.3692,-99.2909
34,7000,Gustavo A. Madero,Aragón la Villa,19.4787,-99.1147
35,8000,Iztacalco,Gabriel Ramos Millán Sección Bramadero,19.3927,-99.0948


In [336]:
def getNearbyNeighborhoods(postalcode):
    
    neighs=[]
    nearby_neighborhoods = df_MetroZone[df_MetroZone['Postal Code']== postalcode]
        
    boro = nearby_neighborhoods['Borough'].values[0]
    lat = nearby_neighborhoods['Latitud'].values[0]
    longitud = nearby_neighborhoods['Longitud'].values[0]
    
    for index in nearby_neighborhoods.index:
        neighs.append(nearby_neighborhoods['Neighborhood'][index])    
        
    conneigh = ', '.join(neighs)
    
    df_data_cp = pd.DataFrame({'Postal Code':postalcode,
                               'Borough':boro, 
                               'Neighborhood':conneigh,
                              'Latitud':lat,
                              'Longitud':longitud}, index=[0])
    
    return df_data_cp

In [337]:
indexDrop = df_data_2[(df_data_2['Postal Code'] == 2000)].index 
df_data_2.drop(indexDrop, inplace=True)

In [338]:
df_data_2.head()

,Postal Code,Borough,Neighborhood,Latitud,Longitud


In [339]:
pcodes = df_MetroZone['Postal Code'].unique()

for pcode in pcodes: 
    df_data_cp = getNearbyNeighborhoods(postalcode=pcode)
    df_data_2 = df_data_2.append(df_data_cp, ignore_index = True)

In [340]:
#df_data_2.sort_values(by=['Postal Code'])
df_data_2

,Postal Code,Borough,Neighborhood,Latitud,Longitud
0,2000,Azcapotzalco,Centro de Azcapotzalco,19.4815,-99.1862
1,4000,Coyoacán,Villa Coyoacán,19.3400,-99.1617
2,5000,Cuajimalpa de Morelos,Cuajimalpa,19.3692,-99.2909
3,7000,Gustavo A. Madero,Aragón la Villa,19.4787,-99.1147
4,8000,Iztacalco,Gabriel Ramos Millán Sección Bramadero,19.3927,-99.0948
5,9000,Iztapalapa,"La Asunción, San Ignacio, San José, San Pablo,...",19.3491,-99.0569
6,10000,La Magdalena Contreras,Lomas Quebradas,19.3155,-99.2406
7,12000,Milpa Alta,"Santa Martha, Villa Milpa Alta Centro, Santa C...",19.3014,-99.0987
8,1000,Álvaro Obregón,San Ángel,19.3587,-99.2033
9,13000,Tláhuac,La Asunción,19.2770,-99.0028


In [341]:
address = 'Distrito Federal, MX'

geolocator = Nominatim(user_agent="df_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Mexico City are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Mexico City are 19.320556250000003, -99.15170107077653.


In [342]:
# create map of CDMX using latitude and longitude values

cdmx_data = df_data_2
map_cdmx = folium.Map(location=[latitude, longitude], zoom_start=11)

# add markers to map
for lat, lng, label in zip(cdmx_data['Latitud'], cdmx_data['Longitud'], cdmx_data['Neighborhood']):
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_cdmx)  
    
map_cdmx

In [343]:
# The code was removed by Watson Studio for sharing.

**Let's explore the 14th neighborhood in our dataframe (Miguel Hidalgo)**

In [344]:
cdmx_data.loc[1, 'Neighborhood']


'Villa Coyoacán'

In [345]:
neighborhood_latitude = cdmx_data.loc[1, 'Latitud'] # neighborhood latitude value
neighborhood_longitude = cdmx_data.loc[1, 'Longitud'] # neighborhood longitude value

neighborhood_name = cdmx_data.loc[1, 'Neighborhood'] # neighborhood name

print('Latitude and longitude values of {} are {}, {}.'.format(neighborhood_name, 
                                                               neighborhood_latitude, 
                                                               neighborhood_longitude))

Latitude and longitude values of Villa Coyoacán are 19.34, -99.1617.


In [346]:
LIMIT = 100 # limit of number of venues returned by Foursquare API

radius = 500 # define radius

# create URL
url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
    CLIENT_ID, 
    CLIENT_SECRET, 
    VERSION, 
    neighborhood_latitude, 
    neighborhood_longitude, 
    radius, 
    LIMIT)
url # display URL


'https://api.foursquare.com/v2/venues/explore?&client_id=NT32XWMDTVGOATILOZHP0G0R4LVAFFZLPV0MD0HJE3EFCVWO&client_secret=MVMRT33IOJWX4U44ALELFFXX0IAX5DITHNMEUNGPCGIXQRHX&v=20180605&ll=19.34,-99.1617&radius=500&limit=100'

In [347]:
results = requests.get(url).json()
results

{'meta': {'code': 200, 'requestId': '5ebb5bed542890001baf3c7b'},
 'response': {'headerLocation': 'Coyoacán',
  'headerFullLocation': 'Coyoacán',
  'headerLocationGranularity': 'city',
  'totalResults': 4,
  'suggestedBounds': {'ne': {'lat': 19.344500004500006,
    'lng': -99.15693977810176},
   'sw': {'lat': 19.335499995499994, 'lng': -99.16646022189823}},
  'groups': [{'type': 'Recommended Places',
    'name': 'recommended',
    'items': [{'reasons': {'count': 0,
       'items': [{'summary': 'This spot is popular',
         'type': 'general',
         'reasonName': 'globalInteractionReason'}]},
      'venue': {'id': '4c8abccae51e6dcbc4cd6bde',
       'name': 'El Tajín',
       'location': {'address': 'Centro Cultural Veracruzano',
        'lat': 19.344045827038187,
        'lng': -99.16316380633484,
        'labeledLatLngs': [{'label': 'display',
          'lat': 19.344045827038187,
          'lng': -99.16316380633484}],
        'distance': 475,
        'cc': 'MX',
        'city': 'Ci

In [348]:
# function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']


In [349]:
venues = results['response']['groups'][0]['items']
    
nearby_venues = json_normalize(venues) # flatten JSON

# filter columns
filtered_columns = ['venue.name', 'venue.categories', 'venue.location.lat', 'venue.location.lng']
nearby_venues =nearby_venues.loc[:, filtered_columns]

# filter the category for each row
nearby_venues['venue.categories'] = nearby_venues.apply(get_category_type, axis=1)

# clean columns
nearby_venues.columns = [col.split(".")[-1] for col in nearby_venues.columns]

nearby_venues.head()

,name,categories,lat,lng
0,El Tajín,Mexican Restaurant,19.344046,-99.163164
1,The Green Corner,Vegetarian / Vegan Restaurant,19.344169,-99.161397
2,Café El Jarocho,Café,19.344178,-99.160771
3,Upper East Side,Wine Bar,19.336676,-99.164041


In [350]:
print('{} venues were returned by Foursquare.'.format(nearby_venues.shape[0]))

4 venues were returned by Foursquare.


**Explore Neighborhoods in CDMX**

In [351]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [352]:
cdmx_venues = getNearbyVenues(names=cdmx_data['Neighborhood'],
                                   latitudes=cdmx_data['Latitud'],
                                   longitudes=cdmx_data['Longitud']
                                  )

Centro de Azcapotzalco
Villa Coyoacán
Cuajimalpa
Aragón la Villa
Gabriel Ramos Millán Sección Bramadero
La Asunción, San Ignacio, San José, San Pablo, San Lucas, San Pedro, Santa Bárbara
Lomas Quebradas
Santa Martha, Villa Milpa Alta Centro, Santa Cruz, La Concepción, La Luz, San Mateo, Los Ángeles
San Ángel
La Asunción
Tlalpan Centro, Tlalpan
Barrio San Antonio, Barrio San Juan, Barrio La Concepción Tlacoapa
Piedad Narvarte
Centro (Área 1)
Lomas de Chapultepec III Sección, Lomas de Chapultepec IV Sección, Lomas de Chapultepec VIII Sección, Lomas de Chapultepec V Sección, Lomas de Chapultepec VI Sección, Lomas de Chapultepec II Sección, Lomas de Chapultepec VII Sección, Lomas de Chapultepec I Sección
General Ignacio Zaragoza
San Cristóbal Centro, Ecatepec Centro
Lerma de Villada Centro
San Bartolo Naucalpan (Naucalpan Centro)
Benito Juárez (La Aurora)
Tezoyuca
Alta Vista, Tlalnepantla  Centro
Centro
Villa de Allende


In [353]:
print(cdmx_venues.shape)
cdmx_venues.head()

(517, 7)


,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,Centro de Azcapotzalco,19.4815,-99.1862,Casa de la Cultura de Azcapotzalco,19.480523,-99.186058,Art Gallery
1,Centro de Azcapotzalco,19.4815,-99.1862,La Conchería CDMX,19.483789,-99.185843,Bakery
2,Centro de Azcapotzalco,19.4815,-99.1862,Antojitos Elsa,19.482932,-99.187686,Mexican Restaurant
3,Centro de Azcapotzalco,19.4815,-99.1862,La Perla Tapatía,19.483741,-99.185856,Mexican Restaurant
4,Centro de Azcapotzalco,19.4815,-99.1862,"Carnitas ""El cortijo""",19.481427,-99.184408,Taco Place


In [354]:
cdmx_venues.groupby('Neighborhood').count()

,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighborhood,,,,,,
"Alta Vista, Tlalnepantla Centro",46,46,46,46,46,46
Aragón la Villa,17,17,17,17,17,17
"Barrio San Antonio, Barrio San Juan, Barrio La Concepción Tlacoapa",1,1,1,1,1,1
Benito Juárez (La Aurora),3,3,3,3,3,3
Centro,23,23,23,23,23,23
Centro (Área 1),59,59,59,59,59,59
Centro de Azcapotzalco,42,42,42,42,42,42
Cuajimalpa,22,22,22,22,22,22
Gabriel Ramos Millán Sección Bramadero,21,21,21,21,21,21


**Analyze Each Neighborhood**

In [355]:
# one hot encoding
cdmx_onehot = pd.get_dummies(cdmx_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
cdmx_onehot['Neighborhood'] = cdmx_venues['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [cdmx_onehot.columns[-1]] + list(cdmx_onehot.columns[:-1])
cdmx_onehot = cdmx_onehot[fixed_columns]

cdmx_onehot.head()

,Neighborhood,American Restaurant,Antique Shop,Argentinian Restaurant,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,BBQ Joint,...,Sushi Restaurant,Taco Place,Theater,Theme Restaurant,Track,Vegetarian / Vegan Restaurant,Wine Bar,Wings Joint,Women's Store,Yoga Studio
0,Centro de Azcapotzalco,0,0,0,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,Centro de Azcapotzalco,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,Centro de Azcapotzalco,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,Centro de Azcapotzalco,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,Centro de Azcapotzalco,0,0,0,0,0,0,0,0,0,...,0,1,0,0,0,0,0,0,0,0


In [356]:
cdmx_onehot.shape

(517, 133)

In [357]:
cdmx_grouped = cdmx_onehot.groupby('Neighborhood').mean().reset_index()
cdmx_grouped

,Neighborhood,American Restaurant,Antique Shop,Argentinian Restaurant,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,BBQ Joint,...,Sushi Restaurant,Taco Place,Theater,Theme Restaurant,Track,Vegetarian / Vegan Restaurant,Wine Bar,Wings Joint,Women's Store,Yoga Studio
0,"Alta Vista, Tlalnepantla Centro",0.000000,0.000000,0.000000,0.00000,0.021739,0.021739,0.000000,0.000000,0.021739,...,0.000000,0.021739,0.021739,0.000000,0.000000,0.021739,0.00,0.021739,0.000000,0.000000
1,Aragón la Villa,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.058824,0.294118,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000
2,"Barrio San Antonio, Barrio San Juan, Barrio La...",0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000
3,Benito Juárez (La Aurora),0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.333333,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000
4,Centro,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.304348,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000
5,Centro (Área 1),0.000000,0.000000,0.016949,0.00000,0.016949,0.000000,0.000000,0.000000,0.016949,...,0.016949,0.067797,0.016949,0.000000,0.000000,0.016949,0.00,0.000000,0.000000,0.000000
6,Centro de Azcapotzalco,0.000000,0.000000,0.000000,0.02381,0.000000,0.000000,0.000000,0.000000,0.023810,...,0.023810,0.095238,0.000000,0.000000,0.000000,0.000000,0.00,0.023810,0.000000,0.000000
7,Cuajimalpa,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,0.045455,0.000000,0.000000,...,0.000000,0.227273,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000
8,Gabriel Ramos Millán Sección Bramadero,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,0.095238,...,0.000000,0.142857,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000
9,General Ignacio Zaragoza,0.000000,0.025641,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.256410,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000


In [358]:
cdmx_grouped.shape

(22, 133)

In [359]:
num_top_venues = 5

for hood in cdmx_grouped['Neighborhood']:
    print("----"+hood+"----")
    temp = cdmx_grouped[cdmx_grouped['Neighborhood'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----Alta Vista, Tlalnepantla  Centro----
                  venue  freq
0    Mexican Restaurant  0.11
1    Seafood Restaurant  0.09
2                   Bar  0.09
3                Bakery  0.07
4  Fast Food Restaurant  0.04


----Aragón la Villa----
                      venue  freq
0                Taco Place  0.29
1        Mexican Restaurant  0.18
2                 Juice Bar  0.06
3  Mediterranean Restaurant  0.06
4            Sandwich Place  0.06


----Barrio San Antonio, Barrio San Juan, Barrio La Concepción Tlacoapa----
                      venue  freq
0                       Bar   1.0
1       American Restaurant   0.0
2                     Motel   0.0
3  Mediterranean Restaurant   0.0
4               Men's Store   0.0


----Benito Juárez (La Aurora)----
                      venue  freq
0                Taco Place  0.33
1                    Bakery  0.33
2                     Plaza  0.33
3  Mediterranean Restaurant  0.00
4               Men's Store  0.00


----Centro----
           

In [360]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

**Top 10 venues for each neighborhood**

In [361]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = cdmx_grouped['Neighborhood']

for ind in np.arange(cdmx_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(cdmx_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.head()

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,"Alta Vista, Tlalnepantla Centro",Mexican Restaurant,Bar,Seafood Restaurant,Bakery,Pharmacy,Public Art,Fast Food Restaurant,Breakfast Spot,Diner,Pizza Place
1,Aragón la Villa,Taco Place,Mexican Restaurant,Bar,Museum,Juice Bar,Pizza Place,Sushi Restaurant,Mediterranean Restaurant,Sandwich Place,Spiritual Center
2,"Barrio San Antonio, Barrio San Juan, Barrio La...",Bar,Yoga Studio,Diner,Event Space,Drugstore,Donut Shop,Dog Run,Dive Bar,Dessert Shop,Farmers Market
3,Benito Juárez (La Aurora),Plaza,Bakery,Taco Place,Yoga Studio,Department Store,Donut Shop,Dog Run,Dive Bar,Diner,Dessert Shop
4,Centro,Taco Place,Seafood Restaurant,Mexican Restaurant,Department Store,Park,Pharmacy,Dessert Shop,Shopping Mall,Coffee Shop,Diner


**Cluster Neighborhoods**

In [380]:
# set number of clusters
kclusters = 5

cdmx_grouped_clustering = cdmx_grouped.drop('Neighborhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(cdmx_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10]

array([1, 0, 3, 4, 0, 1, 1, 0, 1, 0], dtype=int32)

In [386]:
# add clustering labels
#neighborhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

cdmx_merged = cdmx_data

# merge cdmx_grouped with cdmx_data to add latitude/longitude for each neighborhood
cdmx_merged = cdmx_merged.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on='Neighborhood')

cdmx_merged.head() # check the last columns!

,Postal Code,Borough,Neighborhood,Latitud,Longitud,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,2000,Azcapotzalco,Centro de Azcapotzalco,19.4815,-99.1862,1.0,Mexican Restaurant,Taco Place,Bakery,Ice Cream Shop,Seafood Restaurant,Steakhouse,Breakfast Spot,Coffee Shop,Cajun / Creole Restaurant,Wings Joint
1,4000,Coyoacán,Villa Coyoacán,19.3400,-99.1617,1.0,Café,Wine Bar,Vegetarian / Vegan Restaurant,Mexican Restaurant,Yoga Studio,Dog Run,Dive Bar,Diner,Dessert Shop,Department Store
2,5000,Cuajimalpa de Morelos,Cuajimalpa,19.3692,-99.2909,0.0,Taco Place,Mexican Restaurant,Convenience Store,Gym,Farmers Market,Other Great Outdoors,Asian Restaurant,Pizza Place,Ice Cream Shop,Candy Store
3,7000,Gustavo A. Madero,Aragón la Villa,19.4787,-99.1147,0.0,Taco Place,Mexican Restaurant,Bar,Museum,Juice Bar,Pizza Place,Sushi Restaurant,Mediterranean Restaurant,Sandwich Place,Spiritual Center
4,8000,Iztacalco,Gabriel Ramos Millán Sección Bramadero,19.3927,-99.0948,1.0,Bakery,Taco Place,BBQ Joint,Pizza Place,Sandwich Place,Café,Market,Fast Food Restaurant,Flea Market,Bar


In [394]:
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

cdmx_merged['Cluster Labels'].fillna(0, inplace=True)

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(cdmx_merged['Latitud'], cdmx_merged['Longitud'], cdmx_merged['Neighborhood'], cdmx_merged['Cluster Labels'].astype(int)):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters